In [19]:
require 'nn'
require 'rnn'
include '/Users/allanjie/Documents/workspace/statnlp-ner/nn-crf-interface/neural_server/SeqBRNNGRU.lua'
torch.manualSeed(1234)

In [20]:
embeddingSize = 3
vocabSize = 5
numSents = 2
numWords = 3

In [40]:
lt = nn.LookupTableMaskZero(vocabSize, embeddingSize)
function buildBiGRU(inputSize, outputSize, dropout, tanhGRU)
    local bigru = nn.Sequential():add(nn.Transpose({1,2})):add(nn.SplitTable(1))
    local fwdSeq = nn.Sequencer(nn.GRU(inputSize, outputSize, 9999, dropout):maskZero(1))
    local bwdSeq = nn.Sequential():add(nn.ReverseTable())
    bwdSeq:add(nn.Sequencer(nn.GRU(inputSize, outputSize, 9999, dropout):maskZero(1)))
    bwdSeq:add(nn.ReverseTable())
    local biconcat = nn.ConcatTable():add(fwdSeq):add(bwdSeq)
    bigru:add(biconcat):add(nn.ZipTable()):add(nn.Sequencer(nn.JoinTable(1,1)))
    local mapTable = nn.MapTable()
    local combineSize
    if tanhGRU then
        local mapOp = nn.Sequential()
        mapOp:add(nn.Linear(2 * outputSize, outputSize)):add(nn.Tanh())
        mapOp:add(nn.Unsqueeze(1))
        mapTable:add(mapOp)
        combineSize = outputSize
    else
        mapTable:add(nn.Unsqueeze(1))
        combineSize = 2 * outputSize
    end
    bigru:add(mapTable):add(nn.JoinTable(1)):add(nn.Transpose({1,2}))
    return bigru
end

brnn = nn.SeqBRNNGRU(embeddingSize, embeddingSize, true, nn.JoinTable(3))
brgru = buildBiGRU(embeddingSize, embeddingSize, 0.0, true)

In [41]:
input = torch.IntTensor(numSents,numWords)
input[1][1] = 2
input[1][2] = 1
input[1][3] = 4
input[2][1] = 3
input[2][2] = 2
input[2][3] = 0
print(input)

 2  1  4
 3  2  0
[torch.IntTensor of size 2x3]



In [42]:
ylt = lt:forward(input)
-- print(ylt)
-- ybrnn = 
ybrnn = brgru:forward(ylt)
print(ybrnn)

(1,.,.) = 
  0.2241 -0.0318 -0.3360
  0.0161 -0.0836 -0.3217
 -0.2200 -0.1084 -0.3217

(2,.,.) = 
  0.2043 -0.0514 -0.4028
  0.1308  0.0018 -0.3930
 -0.1242 -0.1052 -0.3834
[torch.DoubleTensor of size 2x3x3]

